# Professional Athlete Classifier Using Azure Machine Learning Service 

The following tutorial demonstrates an end-to-end workflow for training and deploying a **Tensorflow 2.0** model on Azure Machine Learning Service. The notebook will cover the following topics:

- Azure Machine Learning service workspace and basic concepts
- Creating a dataset using Azure Cognitive Services and hosting it on a blob datastore
- Setting up an auto-scaling AML Compute cluster
- Training a distributed model across multiple nodes and GPUs
- Deploying a model and creating a web service endpoint


## Problem Statement

Professional sporting events have the power to bring friends, families, and co-workers together at home, at bars, and at the venue itself. While most people enjoy the social aspect and the entertainment of the game, few can recognize every single athlete that is on the court, field, or ice.

In this demo, we will be building a model that help solves this problem by classifying NBA players based upon their photo. While this can be extended to a multi-classification problem, this example will focus on a binary classification between two of the top NBA players today - Lebron James and Stephen Curry.

<table>
    <tr>
        <td>
            <h2>Lebron James</h2>
            <img src='./test/lebron.jpg' height='200' width='400'>
        </td>
        <td>
            <h2>Stephen Curry</h2>
            <img src='./test/steph.jpg' height='300' width='475'>
        </td>
    </tr>
</table>

## What is Azure Machine Learning Service?
Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](./images/aml-overview.png)

## Prerequisites

The tutorial requires the following prerequisites:

- An existing Azure Machine Learning service workspace
- An existing notebook VM instance inside the workspace

The tutorial is designed to be run on the notebook VM instance. Click [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) for instructions on creating a workspace and notebook VM.

## Connect To Workspace

The [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. The workspace holds all your experiments, compute targets, models, datastores, etc.

You can [open ml.azure.com](https://ml.azure.com) to access your workspace resources through a graphical user interface of **Azure Machine Learning studio**.

![](./images/aml-workspace.png)

**You will be asked to login in the next step. Use your Microsoft AAD credentials.**

In [ ]:
from azureml.core import Workspace

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

## Create Dataset Using Azure Cognitive Services

[Azure Cognitive Services](https://azure.microsoft.com/en-us/services/cognitive-services/) is a comprehensive family of AI services and cognitive APIs. We will be using the [Bing Image Search API](https://azure.microsoft.com/en-us/services/cognitive-services/bing-image-search-api/) to find and download images of professional athletes in order to build a dataset that can be used for training.

![](./images/cog-services.png)

To complete this section, you will need to create an Azure Cognitive Service resource in order to obtain an API key. [Click here](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Cwindows) for instructions on how to do so.

**Save the API key in a variable that we will use later.**

In [ ]:
cognitive_service_api_key = '<API-KEY-HERE>'

We have prepared a script which does the following steps in sequence:

1. For each athlete, make a query to the Bing Image Search API for profile photos.
2. Download each image from the resulting response and crop out the face of the athlete.
3. Save the images to a data directory in corresponding test/valid and label folders.

**Let's take a look at the data preparation script**

In [ ]:
%pycat src/cognitive-service-web-scraper.py

**Install the necessary packages to the run the script**

In [ ]:
%pip install opencv-python imutils

The script downloads images in a predefined directory. In order to make it easier for training later on, we will mount a [Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/) container into our local directory and download the images directly into that datastore.

When an Azure Machine Learning workspace is created, a datastore is created by default.

**Get the default datastore of the workspace**

In [ ]:
# Get default datastore
datastore = workspace.get_default_datastore()

**Create a blobfuse config file which is required for mounting**

In [ ]:
config_file_name = 'blobfuse_connection.cfg'

config_params = 'accountName {}\naccountKey {}\ncontainerName {}'.format(
    datastore.account_name, 
    datastore.account_key, 
    datastore.container_name)

config_file = open(config_file_name, 'w')
config_file.writelines(config_params)
config_file.close()

**Create directory that will map to the Blob Storage container**

In [ ]:
blobstore_dir = 'blobstore'

In [ ]:
import os
if not os.path.exists(blobstore_dir):
    os.makedirs(blobstore_dir)

**Mount the datastore using blobfuse**

In [ ]:
!sudo blobfuse {blobstore_dir} --tmp-path=/mnt/resource/blobfusetmp  --config-file={config_file_name} -o attr_timeout=240 -o entry_timeout=240 -o negative_timeout=120

**Now we can run the script to build our dataset**

In [ ]:
data_dir = 'nba-players'

In [ ]:
!sudo mkdir {blobstore_dir}/{data_dir}

In [ ]:
import sys
!sudo {sys.executable} cognitive-service-web-scraper.py --root_dir {blobstore_dir}/{data_dir} --image_dim 400 --api_key {cognitive_service_api_key}

## Register Dataset

Now we can register the directory in our datastore as a [Dataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py), which is simply a reference to a file or folder in a storage container. Registered datasets can then be mounted to compute targets during training making it very easy to have your data be accessible on any machine during training.

In [ ]:
from azureml.core import Dataset

nba_players_dataset = Dataset.File.from_files(path=(datastore, data_dir))

nba_players_dataset = nba_players_dataset.register(workspace=workspace,
                                       name='NBA Players Dataset',
                                       description='Dataset containing pictures of NBA players')

**If the dataset has already been registered, then you (and other users in your workspace) can directly run this cell to get access to it.**

In [ ]:
nba_players_dataset = workspace.datasets['NBA Players Dataset']

## Create Compute Target

A [compute target](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.computetarget?view=azure-ml-py) is a designated compute resource/environment where you run your training script or host your service deployment. This location may be your local machine or a cloud-based compute resource. Compute targets can be reused across the workspace for different runs and experiments. 

For this tutorial, we will create an auto-scaling [Azure Machine Learning Compute](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute?view=azure-ml-py) cluster, which is a managed-compute infrastructure that allows the user to easily create a single or multi-node compute. To create the cluster, we need to specify the following parameters:

- `vm_size`: The is the type of GPUs that we want to use in our cluster. For this tutorial, we will use **Standard_NC12 (NVIDIA K80) GPU Machines** .
- `idle_seconds_before_scaledown`: This is the number of seconds before a node will scale down in our auto-scaling cluster. We will set this to **6000** seconds. 
- `min_nodes`: This is the minimum numbers of nodes that the cluster will have. To avoid paying for compute while they are not being used, we will set this to **0** nodes.
- `max_modes`: This is the maximum number of nodes that the cluster will scale up to. Will will set this to **4** nodes.

**When jobs are submitted to the cluster it takes approximately 5 minutes to allocate new nodes** 

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = 'k80cluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                       idle_seconds_before_scaledown=1200,
                                                       min_nodes=0, 
                                                       max_nodes=2)

compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

**If the compute target has already been created, then you (and other users in your workspace) can directly run this cell to get access to it.**

In [ ]:
compute_target = workspace.compute_targets['k80cluster']

## Perform Experiment

Now that we have our dataset registered and compute target created, it is time to start training our model. We will start by creating an [experiment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py). An experiment is a grouping of many runs from a specified script. All runs in this tutorial will be performed under the same experiment. 

In [ ]:
from azureml.core import Experiment

experiment_name = 'nba-player-classifier' 
experiment = Experiment(workspace, name=experiment_name)

A training script has been prepared in advance and will be used to train our model. The Tensorflow 2.0 training script adds dropout and classification layers to a base ResNet50 model and then fine-tunes the weights on our new dataset.

**Let's take a look at what the script looks like exactly**

In [ ]:
%pycat src/train.py

**Next step is to create a TensorFlow Estimator**

The Azure Machine Learning Python SDK Estimator classes allow you to easily construct run configurations for your experiments. They allow you too define parameters such as the training script to run, the compute target to run it on, framework versions, additional package requirements, etc. 

You can also use a generic [Estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py) to submit training scripts that use any learning framework you choose.

For popular libaries like PyTorch and Tensorflow you can use their framework specific estimators. We will use the [TensorFlow Estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn.tensorflow?view=azure-ml-py) for our experiment.

In [ ]:
from azureml.train.dnn import TensorFlow

estimator = TensorFlow(source_directory='src',
                                  entry_script='train.py',
                                  compute_target=compute_target,
                                  script_params = {
                                      '--data_dir': nba_players_dataset.as_named_input('nbaplayersdata').as_mount(),
                                      '--image_dim': 250,
                                      '--learning_rate': 0.001,
                                      '--batch_size': 16,
                                      '--steps_per_epoch': 100,
                                      '--num_epochs': 25,
                                      '--dropout_rate': 0.5,
                                      '--activation_function': 'softmax',
                                      '--output_dir':'./outputs',
                                  },
                                  framework_version='2.0',
                                  use_gpu=True,
                                  pip_packages=['Pillow==6.2.0', 'scipy==1.1.0', 'azureml-dataprep[fuse,pandas]==1.1.29'])

**Now we submit our estimator to the experiment**

In [ ]:
run = experiment.submit(estimator)

**We can monitor the progress of our run within the notebook**

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

**Now we can register the trained model into our workspace**

In [ ]:
model = run.register_model(model_name='nba-player-classifier', 
                           model_path='outputs',
                           datasets=[('train, test, validation data', nba_players_dataset)],
                           description='ResNet 50 model of for classifying NBA players')

## Deploy Model

Once we are satisified with the performance of our model, we can deploy it as a web service. For this tutorial, we will be deploying our service to an [Azure Container Instance (ACI)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.aciwebservice?view=azure-ml-py). Azure Machine Learning has options for [local](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.localwebservice?view=azure-ml-py) and [Azure Kubernetes Service (AKS)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py) deployment as well. Deployment of a model requires the following dependencies:

- A scoring script to show how to use the model
- An environment file to show what packages need to be installed
- A configuration file to build the web service
- The model that was previously trained

The scoring script requires two methods:

- **init()**: This method is run upon deployment of the web service container. This is typically where the model is loaded.
- **run()**: This method is run each time a request is sent to the web service. This is your predict method.

**Let's take a look at the scoring script that has already been created.**

In [ ]:
%pycat src/score.py

To assist with creating the environment file, we will be using the [CondaDepencies](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) class to define the required python packages for running our scoring script. 

**The code below creates the environment file that we need for deployment.**

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','pandas'],
                                 pip_packages=['inference-schema[numpy-support]', 'azureml-defaults', 'tensorflow==2.0.0', 'opencv-python==4.1.1.26'])

with open('src/myenv.yml','w') as f:
    f.write(myenv.serialize_to_string())

**Let's take a look at the environment file that was created**

In [ ]:
%pycat src/myenv.yml

**We define our deployment configuration using the [InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) class.**

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='src',
                                   runtime= 'python', 
                                   entry_script='score.py',
                                   conda_file='myenv.yml')

**Now we can create and deploy our web service (deployment will take about 10 minutes).**

In [ ]:
from azureml.core import Model
model = Model(workspace, name='nba-player-classifier')

In [ ]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, 
                                                memory_gb=4, 
                                                tags={'model': 'RESNET 50', 'method': 'tensorflow'}, 
                                                description='Predicts NBA player based on image')

aci_service_name = 'nba-player-aciservice'

try:
    aci_service = Webservice(workspace, name=aci_service_name)
    if aci_service:
        aci_service.delete()
except WebserviceException as e:
    print()

In [ ]:
aci_service = Model.deploy(workspace, aci_service_name, [model], inference_config, aci_config)

aci_service.wait_for_deployment(True)
print(aci_service.state)

## Test Web Service

Once the model has been successfully deployed, we can test the web service.

**We can easily check out the HTTP endpoint of the web service.**

In [ ]:
from azureml.core.webservice import Webservice
aci_service = Webservice(workspace, name='nba-player-aciservice')

In [ ]:
print(aci_service.scoring_uri)

**Send in some requests to test our service.**

In [ ]:
import requests

test_images = [
    'https://a.espncdn.com/photo/2019/0603/r551314_1296x729_16-9.jpg', 
    'https://media.voltron.voanews.com/Drupal/01live-166/2019-04/00F04247-6B94-4259-AE2E-916E333E9C79.jpg'
]

predictions = []

for url in test_images:
    input_data = '{\"image_url\": \"'+ url +'\"}'
    headers = {'Content-Type':'application/json'}
    response = requests.post(aci_service.scoring_uri, input_data, headers=headers)
    predictions.append(response)

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
from matplotlib import rcParams
%matplotlib inline

labels = ['Lebron James', 'Stephen Curry']

# Download images
img_A = Image.open(BytesIO(requests.get(test_images[0]).content))
img_B = Image.open(BytesIO(requests.get(test_images[1]).content))

# Get predictions
result_A = np.array(json.loads(json.loads(predictions[0].text))['prediction'][0])
result_B = np.array(json.loads(json.loads(predictions[1].text))['prediction'][0])
prediction_A = 'Prediction: {}\n Probability: {}'.format(labels[result_A.argmax()], result_A.max())
prediction_B = 'Prediction: {}\n Probability: {}'.format(labels[result_B.argmax()], result_B.max())

# Display images
rcParams['figure.figsize'] = 20,15
fig, ax = plt.subplots(1,2)
ax[0].set_title(prediction_A)
ax[0].imshow(img_A)
ax[1].set_title(prediction_B)
ax[1].imshow(img_B)